### Trajectory Preprocessing
Here we plot each trial trajectory to see if there are any outliers. We can see that there are some trajectories that are very different from the rest. We will remove these trajectories from the dataset. Then we save it to a new dataframe

In [ ]:
df_manrev = pd.read_json(r"C:\Users\veoni\Documents\GitHub\MTE_DLAB\Analysis\DF_manrev.json")
#df_manrev = pd.DataFrame(columns=['pt_num', 'trial_nbr', 'block_number', 'acc', 'choice', 'true_value', 'difficulty', 'rt', 'previous_response', 'downsampled_traj', 'timestamps'])
class TrialViewer:
    def __init__(self, df_filtered):
        self.df_filtered = df_filtered
        self.current_index = 48001

        self.root = tk.Tk()
        self.root.title("Trial Viewer")

        self.label = tk.Label(self.root, text="Trial: 0")
        self.label.pack()

        self.figure, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.root)
        self.canvas.get_tk_widget().pack()

        self.reject_button = tk.Button(self.root, text="Reject (F)", command=self.reject_trial)
        self.reject_button.pack(side=tk.LEFT)
        self.reject_button.bind("<F>", self.reject_trial)  # Bind 'F' key

        self.accept_button = tk.Button(self.root, text="Accept (J)", command=self.accept_trial)
        self.accept_button.pack(side=tk.LEFT)
        self.accept_button.bind("<J>", self.accept_trial)  # Bind 'J' key

        self.previous_button = tk.Button(self.root, text="Previous", command=self.previous_trial)
        self.previous_button.pack(side=tk.LEFT)

        self.next_button = tk.Button(self.root, text="Next", command=self.next_trial)
        self.next_button.pack(side=tk.LEFT)

        self.show_current_trial()
        
        # Set focus on the root window to capture keyboard events
        self.root.focus_set()

        # Bind keyboard shortcuts
        self.root.bind("<Key>", self.key_pressed)

        self.root.mainloop()

    def show_current_trial(self):
        trial_data = self.df_filtered.iloc[self.current_index]['downsampled_traj']
        pt_num = self.df_filtered.iloc[self.current_index]['pt_num']  # Get pt_num
        choice = self.df_filtered.iloc[self.current_index]['choice']  
        trial_nbr = self.df_filtered.iloc[self.current_index]['trial_nbr']
        self.ax.clear()
        x_coords = [pos[0] for pos in trial_data]
        y_coords = [pos[1] for pos in trial_data]
        if choice == 'Red':
            self.ax.plot(x_coords, y_coords, color='red')  # Plot each segment in red
        elif choice == 'Blue':
            self.ax.plot(x_coords, y_coords, color='blue')  # Plot each segment in blue
        self.ax.set_xlim(-960, 960)
        self.ax.set_ylim(-200, 600)
        self.ax.hlines(-108, -960, 960, color='orange')  # Add a vertical line at the center
        self.ax.set_title("Trial {} - pt_num: {} - index: {}".format(trial_nbr, pt_num, self.current_index))  # Update title
        self.canvas.draw_idle()
        self.label.config(text="Trial: {}".format(self.current_index))

    def reject_trial(self, event=None):
        # Implement what happens when the trial is rejected
        print("Index {} rejected.".format(self.current_index))
        self.next_trial()

    def accept_trial(self, event=None):
        # Implement what happens when the trial is accepted
        print("Index {} accepted.".format(self.current_index))
        # Add accepted trial to your new dataframe (e.g., df_manrev)
        accepted_trial = self.df_filtered.iloc[self.current_index]
        # Do something with accepted_trial, e.g., 
        global df_manrev
        df_manrev = df_manrev._append(accepted_trial, ignore_index=True)

        # Show the next trial
        self.next_trial()

    def previous_trial(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_current_trial()
            df_manrev = df_manrev.drop(df_manrev.index[-1])

    def next_trial(self):
        if self.current_index < len(self.df_filtered) - 1:
            self.current_index += 1
            self.show_current_trial()
    def key_pressed(self, event):
            if event.char.lower() == 'f':
                self.reject_trial()
            elif event.char.lower() == 'j':
                self.accept_trial()
# Example usage:
# Assuming df_filtered is your filtered DataFrame
# You need to have matplotlib installed for plotting
app = TrialViewer(df_filtered)
#print(TrialViewer.self.current_index)
#df_manrev.to_json(r"C:\Users\veoni\Documents\GitHub\MTE_DLAB\Analysis\DF_manrev.json")

In [ ]:
# Additional trials that need to be removed.
drop_list = [500, 792, 987, 8109, 9204, 9704, 10460, 11225, 11254, 11808, 12171, 14611, 14935,14936, 15470, 15907, 18456, 18499, 19501, 19657, 19869, 20339, 21963, 22423, 22762, 22895, 23870, 23881, 24056, 24417, 24725, 24898, 25756, 26634, 27029, 27630, 27726, 27754, 27889, 28133, 28166, 28290, 28752, 28922, 29195, 29733, 31190, 33168, 33545, 34276, 34517, 34886, 35121, 35167, 35174, 35550, 35764, 36950, 37236, 37393, 38552, 39427, 39491, 39598, 40498, 41304, 41371, 41858, 41983, 42035, 42232, 42456, 42823, 43226, 43541, 43801, 44006, 44190, 44439, 44440, 44581, 44796, 44814, 45452, 46842, 47021, 47032, 47624, 47716, 47791, 47919, 48087, 48114, 48455, 48896, 49011, 49045, 49171, 49458, 50214]
drop_info = []
for i in drop_list:
    drop_info.append((df_filtered['pt_num'].iloc[i], df_filtered['trial_nbr'].iloc[i]))
    
# Drop rows from df_manrev based on pt_num and trial_nbr
drop_info = [(pt_num, trial_nbr) for pt_num, trial_nbr in drop_info]
df_manrev = df_manrev[~df_manrev[['pt_num', 'trial_nbr']].apply(tuple, axis=1).isin(drop_info)]

df_manrev.to_json(r"C:\Users\veoni\Documents\GitHub\MTE_DLAB\Analysis\DF_manrev.json")

After this stage is finished you can return back to the main notebook and load the new dataset, and continue with the rest of the notebook.